# Function optimisation

In [13]:
import numpy as np

from scipy.optimize import minimize
from sympy import *

## Unconstrained minimisation of multivariate scalar functions

### Nelder-Mead algorithm

Rosenbrock function:

$$
f(x,y) = (a - x)^2 + b(y - x^2)^2
$$

In [20]:
def rosen_2d(x):
    """Rosenbrock function."""
    
    a = 1
    b = 100
    
    return (a - x[0])**2 + b*(x[1] -x[0]**2)**2

In [21]:
x0 = np.array([1.3, 0.7])
res = minimize(rosen_2d, x0, method='nelder-mead', 
               options={'xatol':1e-8, 'disp':True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 79
         Function evaluations: 150


In [24]:
# Solution for x
res.x

array([1., 1.])

In [23]:
rosen_2d(res.x)

3.3736077629532093e-18

### Broyden-Fletcher-Goldfarb-Shanno (BFGS) algorithm

Booth function:

$$
f(x,y) = (x + 2y - 7)^2 + (2x + y - 5)^2
$$

First partial derivatives:

$$
\frac{\partial f(x,y)}{\partial x} = 10x + 8y - 34
$$

$$
\frac{\partial f(x,y)}{\partial y} = 8x + 10y - 38
$$

In [17]:
# Check the first partial derivative
x, y = symbols('x y')

print(diff( (x + 2*y -7)**2 + (2*x +y - 5)**2, x ))
print(diff( (x + 2*y -7)**2 + (2*x +y - 5)**2, y ))

10*x + 8*y - 34
8*x + 10*y - 38


In [11]:
def booth(x):
    """Booth function."""
    
    return (x[0] + 2*x[1] - 7)**2 + (2*x[0] + x[1] - 5)**2

In [12]:
def booth_derivative(x):
    """First partial derivative of the Booth function."""
    
    return [10*x[0] + 8*x[1] - 34, 8*x[0] + 10*x[1] - 38]

In [15]:
x0 = [1, 1]

res = minimize(booth, x0, method='BFGS', jac=booth_derivative,
               options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 6
         Function evaluations: 7
         Gradient evaluations: 7


In [16]:
res.x

array([0.99999881, 3.0000011 ])

In [18]:
booth(res.x)

2.654464649283898e-12

### Newton-Conjugate-Gradient algorithm

Rosenbrock function:

$$
f(x,y) = (a - x)^2 + b(y - x^2)^2
$$

In [42]:
# Check the first partial derivative
a, b, f, x, y = symbols('a b f x y')

In [43]:
f = (a-x)**2 + b*(y-x**2)**2

In [56]:
print(diff(f, x))

-2*a - 4*b*x*(-x**2 + y) + 2*x


In [57]:
print(diff(f, y))

b*(-2*x**2 + 2*y)


In [51]:
diff(f, x, x)

2*(4*b*x**2 + 2*b*(x**2 - y) + 1)

In [48]:
diff(f, x, y)

-4*b*x

In [49]:
diff(f, y, y)

2*b

First Partial derivatives:

$$
\frac{\partial f(x,y)}{\partial x} = 2(x - a) - 4bx(y - x^2)
$$

$$
\frac{\partial f(x,y)}{\partial y} = 2b(y-x^2)
$$

Second partial derivatives:

$$
\frac{\partial f^2(x,y)}{\partial x^2} =2(4bx^2 + 2b(x^2 - y) + 1)
$$

$$
\frac{\partial f^2(x,y)}{\partial y \partial x} = -4bx
$$

$$
\frac{\partial f^2(x,y)}{\partial y^2} = 2b
$$

In [62]:
def build_rosen_derivative(a, b):
    """Build a Rosenbrock derivative function."""
    
    def rosen_derivative(x):
        der = np.array([
            -2*a - 4*b*x[0]*(-x[0]**2 + x[1]) + 2*x[0],  # df/dx
            b*(-2*x[0]**2 + 2*x[1])  # df/dy
        ])
        
        return der
        
    return rosen_derivative

In [65]:
def build_rosen_hessian(a, b):

    def rosen_hessian(x):
        """Calculate the Hessian matrix for the 2D Rosenbrock function."""

        H = np.zeros((2,2))
        H[0, 0] = 2*(4*b*x[0]**2 + 2*b*(x[0]**2 - x[1]) + 1)
        H[0, 1] = -4*b*x[0]
        H[1, 0] = -4*b*x[0]
        H[1, 1] = 2*b
        
        return H
        
    return rosen_hessian

In [93]:
a = 1
b = 100

jac = build_rosen_derivative(a, b)
hess = build_rosen_hessian(a, b)

x0 = [1, 1]

res = minimize(rosen_2d, x0, 
               method='Newton-CG', 
               jac=jac, hess=hess,
               options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 1
         Gradient evaluations: 1
         Hessian evaluations: 1


In [94]:
res.x

array([1., 1.])

In [95]:
res.fun

0.0

In [98]:
rosen_2d([1, 1])

0

In [97]:
rosen_2d(res.x)

0.0

## Constrained minimisation of multivariate scalar functions

In [69]:
from scipy.optimize import Bounds

In [83]:
bounds = Bounds([0, -0.5],   # lower
                [1.0, 2.0])  # upper

In [84]:
x0 = [-0.25, 1.5]

res = minimize(booth, x0, method='L-BFGS-B', 
               bounds=bounds,
               jac=booth_derivative,
               options={'disp': True})

In [85]:
res.x

array([1., 2.])